# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

E. Bañados  ->  E. Bañados  |  ['E. Bañados']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
N. Wang  ->  N. Wang  |  ['N. Wang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
C. Clontz  ->  C. Clontz  |  ['C. Clontz']
N. Neumayer  ->  N. Neumayer  |  ['N. Neumayer']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']


X. Zhang  ->  X. Zhang  |  ['X. Zhang']
A. Bayer  ->  M. A. Bayer  |  ['A. Bayer']
Arxiv has 82 new papers today
          9 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/9 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2510.25829


extracting tarball to tmp_2510.25829...

 done.


Issues with the citations
list index out of range
Retrieving document from  https://arxiv.org/e-print/2510.26152


...

extracting tarball to tmp_2510.26561...

 done.
  1: tmp_2510.26561/supplementary_materials.tex, 754 lines
  2: tmp_2510.26561/main.tex, 530 lines
Retrieving document from  https://arxiv.org/e-print/2510.26691


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found 2 candidates with documentclass definition.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Assuming tmp_2510.26561/supplementary_materials.tex as main document.
  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2510.26691...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.25829-b31b1b.svg)](https://arxiv.org/abs/2510.25829) | **Updated dark pixel fraction constraints on reionization's end from the Lyman-series forests of XQR-30**  |
|| F. B. Davies, et al. -- incl., <mark>E. Bañados</mark> |
|*Appeared on*| *2025-10-31*|
|*Comments*| *14+4 pages, 14+6 figures, accepted for publication in MNRAS*|
|**Abstract**|            The fraction of "dark pixels" in the Ly$\alpha$ and other Lyman-series forests at $z\sim 5-6$ provides a powerful constraint on the end of the reionization process. Any spectral region showing transmission must be highly ionized, while dark regions could be ionized or neutral, thus the dark pixel fraction provides a (nearly) model independent upper limit to the volume-filling fraction of the neutral intergalactic medium, modulo choices in binning scale and dark pixel definition. Here we provide updated measurements of the 3.3 comoving Mpc dark pixel fraction at $z=4.85-6.25$ in the Ly$\alpha$, Ly$\beta$, and Ly$\gamma$ forests of 34 deep $5.8\lesssim z\lesssim 6.6$ quasar spectra from the (enlarged) XQR-30 sample. Using the negative pixel method to measure the dark pixel fraction, we derive fiducial $1\sigma$ upper limits on the volume-average neutral hydrogen fraction of $\langle x_{\rm{HI}}\rangle \leq \{0.030 + 0.048, 0.095 + 0.037, 0.191 + 0.056, 0.199 + 0.087\}$ at $\bar{z} = \{5.481, 5.654, 5.831, 6.043\}$ from the optimally sensitive combination of the Ly$\beta$ and Ly$\gamma$ forests. We further demonstrate an alternative method that treats the forest flux as a mixture of dark and transparent regions, where the latter are modeled using a physically-motivated parametric form for the intrinsic opacity distribution. The resulting model-dependent upper limits on $\langle x_{\rm{HI}}\rangle$ are similar to those derived from our fiducial model-independent analysis. We confirm that the bulk of reionization must be finished at $z>6$, while leaving room for an extended "soft landing" to the reionization history down to $z\sim 5.4$ suggested by Ly$\alpha$ forest opacity fluctuations.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.26341-b31b1b.svg)](https://arxiv.org/abs/2510.26341) | **oMEGACat. VIII. A Subpopulation Census of ω Centauri**  |
|| <mark>C. Clontz</mark>, et al. -- incl., <mark>N. Neumayer</mark> |
|*Appeared on*| *2025-10-31*|
|*Comments*| **|
|**Abstract**|            An understanding of the assembly history of the complex star cluster Omega Centauri has long been sought after, with many studies separating the stars on the color-magnitude diagram into multiple groupings across small magnitude ranges. Utilizing the oMEGACat combined astro-photometric and spectroscopic dataset we parse 14 subpopulations from the upper red-giant branch to below the main-sequence turnoff. We combine our results with previous works to estimate the age and age spread of each population. We find that the chemically enhanced (P2) populations are all ~1 Gyr younger (~11.6 Gyr old) and have significantly higher intrinsic age spreads (0.6 Gyr) than the primordial (P1) populations (~12.6 Gyr old, 0.3 Gyr spread), with the intermediate (Im) populations falling in between the two. Additionally, we connect for the first time the Chromosome Diagram to the two-stream age-metallicity relation, allowing us to link the P1 and P2 stars to the distinct star formation tracks, proposed to be in-situ and ex-situ contributions to the cluster's assembly. Our results are consistent with some suggested formation models and rule out others but no current model can explain all observed features of the subpopulations.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.26449-b31b1b.svg)](https://arxiv.org/abs/2510.26449) | **The dance of dust: Investigating young stellar object dipper variability**  |
|| A. Empey, et al. -- incl., <mark>M. Benisty</mark> |
|*Appeared on*| *2025-10-31*|
|*Comments*| *Accepted for publication in Astronomy & Astrophysics (A&A)*|
|**Abstract**|            The dipper subclass of YSOs are characterised by frequent dips in their light curves. Irregular dippers do not show periodic signatures and have dips accounting for significant proportions of their photospheric flux. Given the short timescales on which these dips occur, their driving mechanisms are linked to the inner circumstellar disc dynamics. We present the first multi-epoch analysis of 16 irregular dippers observed with X-Shooter. Investigating the properties of their dips, and in particular the analysis of the dust characteristics, we aim to understand the root of their variability, and get a glimpse of the inner disc behaviour. We employed a novel approach to measure the properties of the dips, by combining class III templates with Gaia photometry to construct the intrinsic photospheres. We measured several dip properties including the depth of the dips, near-infrared (NIR) excesses, and their optical depths as a function of wavelength. We record 20 significant dips that range in their dip properties and show no relation to one another. In almost all cases, the low optical depths and small NIR excesses are observed. Comparison of their optical depths with grain opacity models show that the dips can be explained by the presence of dust substructures containing processed grains obscuring their photospheres and/or their discs. These grain distributions can have maximum sizes as large as 20$\mu m$ and in many cases have almost grey-like extinction, while some require a strong scattering component. The findings highlight the extent of the irregularity of dippers, but also link it to the dust dynamics in the inner regions of circumstellar discs. The dust substructures causing the variability require processed dust grains to be lifted above the disc into the line of sight. Possible lifting mechanisms including disc winds, unstable accretion columns, and disc warps are discussed.         |

## Failed papers

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.26209-b31b1b.svg)](https://arxiv.org/abs/2510.26209) | **Multi-Faceted Emission Properties of PSR J2129+4119 Observed with FAST**  |
|| H. M. Tedila, et al. -- incl., <mark>N. Wang</mark> |
|*Appeared on*| *2025-10-31*|
|*Comments*| *22 pages, 18 figures, Revised for ApJ*|
|**Abstract**|            We present a detailed single-pulse study of the long-period pulsar PSR J2129+4119 using high-sensitivity FAST observations. Despite locating well below the traditional death line, the pulsar exhibits sustained and multi-modal emission behavior, including nulls, weak pulses, regular emission, and occasional bright pulses. The nulling fraction is measured to be $8.13\% \pm 0.51\%$, with null durations typically under four pulse periods. Fluctuation spectral analysis reveals both phase-modulated subpulse drifting and intermittent beat-like modulation. At the same time, polarization profiles show high linear polarization and stable polarization position angle (PPA) swings consistent with a near-tangential sightline geometry. Quasi-periodic microstructures are detected in 11.54\% of regular pulses, with a mean periodicity and width of 4.57 ms and 4.30 ms, respectively. A well-defined scintillation arc in the secondary spectrum confirms the presence of a localized scattering screen. These results indicate that PSR J2129+4119 remains magnetospherically active and coherently emitting despite its low energy loss rate, offering key insights into pulsar emission physics near the death line.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.26250-b31b1b.svg)](https://arxiv.org/abs/2510.26250) | **Theoretical models for the Late Thermal Pulse in post-AGB stars: the case of DY Cen**  |
|| Z. Liu, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-10-31*|
|*Comments*| **|
|**Abstract**|            We present theoretical predictions of the born-again scenario for post-asymptotic giant-branch stars. An extensive model grid for born-again objects has been constructed, particularly including models for the Very Late Thermal Pulse with and without convective overshooting, and also including models for the Late Thermal Pulse. We constructed a large parameter space to analyze the dependencies of the born-again model on core mass, hydrogen-envelope mass, and overshoot parameters, and we analyzed how changes in these parameters affect the models' evolution. We applied our grid of models to interpret observations of DY\,Cen, a star exhibiting characteristics similar to confirmed born-again stars. We compared DY\,Cen with models from multiple aspects, including heating rate, evolutionary tracks, and surface abundances. Ultimately, we concluded that none of our born-again models could match all of the observed properties of DY\,Cen, especially its surface chemistry; DY\,Cen is therefore an unlikely born-again star.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.26355-b31b1b.svg)](https://arxiv.org/abs/2510.26355) | **Model-independent late-universe measurements of $H_0$ and $Ω_\mathrm{K}$ with the PAge-improved inverse distance ladder**  |
|| G.-H. Du, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-10-31*|
|*Comments*| *12 pages, 2 figures*|
|**Abstract**|            The standard $\Lambda{\rm CDM}$ model has encountered serious challenges and the $H_0$ tension has become more significant with increasingly precise cosmological observation. Meanwhile, inconsistencies in measurements of the curvature parameter $\Omega_\mathrm{K}$ between different datasets also have emerged. In this work, we employ two global and cosmic age-based parameterizations, PAge and MAPAge, to perform model-independent measurements of the Hubble constant $H_0$ and $\Omega_\mathrm{K}$ by utilizing the inverse distance ladder (IDL). To construct the PAge-improved IDL, we utilize the strong gravitational lensing (SGL), cosmic chronometers (CC), and gamma ray bursts (GRB) data to calibrate the latest DESI DR2 baryon acoustic oscillation data and DESY5 type Ia supernova data. Our analysis indicate that DESI+DESY5+SGL+CC+GRB gives $H_0=71.59\pm 0.94\,{\rm km}~{\rm s}^{-1}~{\rm Mpc}^{-1}$ in the MAPAge model, reducing the $H_0$ tension to the $1.0\sigma$ level. Extending to MAPAge$+\Omega_{\rm K}$ model, we obtain $\Omega_\mathrm{K}=0.001\pm 0.038$, which suggests that current late-time data are consistent with a flat universe. Finally, the Bayesian analysis indicates that the present late-universe data provide weak to moderate evidence in favor of PAge and MAPAge relative to $\Lambda{\rm CDM}$.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.26561-b31b1b.svg)](https://arxiv.org/abs/2510.26561) | **A Star's Death by a Thousand Cuts: The Runaway Periodic Eruptions of AT2023uqm**  |
|| <mark>Y. Wang</mark>, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-10-31*|
|*Comments*| *Submitted. Comments are welcome*|
|**Abstract**|            Stars on bound orbits around a supermassive black hole may undergo repeated partial tidal disruption events (rpTDEs), producing periodic flares. While several candidates have been suggested, definitive confirmation of these events remains elusive. We report the discovery of AT2023uqm, a nuclear transient that has exhibited at least five periodic optical flares, making it only the second confirmed case of periodicity after ASASSN-14ko. Uniquely, the flares from AT2023uqm show a nearly exponential increase in energy--a "runaway" phenomenon signaling the star's progressive destruction. This behavior is consistent with rpTDEs of low-mass, main-sequence stars or evolved giant stars. Multiwavelength observations and spectroscopic analysis of the two most recent flares reinforce its interpretation as an rpTDE. Intriguingly, each flare displays a similar double-peaked structure, potentially originating from a double-peaked mass fallback rate or two discrete collisions per orbit. The extreme ratio of peak separation to orbital period draws attention to the possibility of a giant star being disrupted, which could be distinguished from a low-mass main-sequence star by its future mass-loss evolution. Our analysis demonstrates the power of rpTDEs to probe the properties of disrupted stars and the physical processes of tidal disruption, though it is currently limited by our knowledge of these events. AT2023uqm emerges as the most compelling rpTDE thus far, serving as a crucial framework for modeling and understanding these phenomena.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.26691-b31b1b.svg)](https://arxiv.org/abs/2510.26691) | **Flinch: A Differentiable Framework for Field-Level Inference of Cosmological parameters from curved sky data**  |
|| A. Crespi, et al. -- incl., <mark>A. Bayer</mark> |
|*Appeared on*| *2025-10-31*|
|*Comments*| *18 pages, 11 figures. Comments are welcome*|
|**Abstract**|            We present Flinch, a fully differentiable and high-performance framework for field-level inference on angular maps, developed to improve the flexibility and scalability of current methodologies. Flinch is integrated with differentiable cosmology tools, allowing gradients to propagate from individual map pixels directly to the underlying cosmological parameters. This architecture allows cosmological inference to be carried out directly from the map itself, bypassing the need to specify a likelihood for intermediate summary statistics. Using simulated, masked CMB temperature maps, we validate our pipeline by reconstructing both maps and angular power spectra, and we perform cosmological parameter inference with competitive precision. In comparison with the standard pseudo-$C_\ell$ approach, Flinch delivers substantially tighter constraints, with error bars reduced by up to 40%. Among the gradient-based samplers routinely employed in field-level analyses, we further show that MicroCanonical Langevin Monte Carlo provides orders-of-magnitude improvements in sampling efficiency over currently employed Hamiltonian Monte Carlo samplers, greatly reducing computational expense.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2510.25829/./xhi_thresh_allforest.png', 'tmp_2510.25829/./threshpix_abg.png', 'tmp_2510.25829/./threshpix_combo.png', 'tmp_2510.25829/./xhi_thresh_fiducial.png', 'tmp_2510.25829/./xhi_negpix_fiducial.png']
copying  tmp_2510.25829/./xhi_thresh_allforest.png to _build/html/
copying  tmp_2510.25829/./threshpix_abg.png to _build/html/
copying  tmp_2510.25829/./threshpix_combo.png to _build/html/
copying  tmp_2510.25829/./xhi_thresh_fiducial.png to _build/html/
copying  tmp_2510.25829/./xhi_negpix_fiducial.png to _build/html/
exported in  _build/html/2510.25829.md
    + _build/html/tmp_2510.25829/./xhi_thresh_allforest.png
    + _build/html/tmp_2510.25829/./threshpix_abg.png
    + _build/html/tmp_2510.25829/./threshpix_combo.png
    + _build/html/tmp_2510.25829/./xhi_thresh_fiducial.png
    + _build/html/tmp_2510.25829/./xhi_negpix_fiducial.png
found figures ['tmp_2510.26341/./figures/sgb_rgb_population_labels.png', 'tmp_2510.26341/./figures/two_panel_amr_age_spread.png'

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\xhi}{\langle x_{\rm HI} \rangle}$
$\newcommand{\noop}[1]$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\newcommand\mn{@urlcharsother}$
$\newcommand\mn{@doi}$
$\newcommand\mn{@doi@}$
$\newcommand\mn{@eprint#1#2}$
$\newcommand\mn{@eprint@arXiv#1}$
$\newcommand\mn{@eprint@dblp#1}$
$\newcommand\mn{@eprint@#1:#2:#3:#4}$
$\newcommand{\@}{tempa}$
$\newcommand{\@}{tempa }$
$\newcommand{\@}{tempb }$
$\newcommand{\@}{tempc$
$  }$
$\newcommand{\@}{tempb }$</div>



<div id="title">

# Updated dark pixel fraction constraints on reionization's end from theLyman-series forestsof XQR-30

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2510.25829-b31b1b.svg)](https://arxiv.org/abs/2510.25829)<mark>Appeared on: 2025-10-31</mark> -  _14+4 pages, 14+6 figures, accepted for publication in MNRAS_

</div>
<div id="authors">

F. B. Davies, et al. -- incl., <mark>E. Bañados</mark>

</div>
<div id="abstract">

**Abstract:** The fraction of "dark pixels" in the Ly $\alpha$ and other Lyman-series forests at $z\sim5$ -- $6$ provides a powerful constraint on the end of the reionization process. Any spectral region showing transmission must be highly ionized, while dark regions could be ionized or neutral, thus the dark pixel fraction provides a (nearly) model independent upper limit to the volume-filling fraction of the neutral intergalactic medium, modulo choices in binning scale and dark pixel definition. Here we provide updated measurements of the 3.3 comoving Mpc dark pixel fraction at $z=4.85$ -- $6.25$ in the Ly $\alpha$ , Ly $\beta$ , and Ly $\gamma$ forests of $34$ deep $5.8 \lesssim z\lesssim6.6$ quasar spectra from the (enlarged) XQR-30 sample. Using the negative pixel method to measure the dark pixel fraction, we derive fiducial $1\sigma$ upper limits on the volume-average neutral hydrogen fraction of $\langle x_{\rm HI} \rangle \leq \{0.030+0.048,0.095+0.037,0.191+0.056,0.199+0.087\}$ at $\bar{z}=\{5.481,5.654,5.831,6.043\}$ from the optimally sensitive combination of the Ly $\beta$ and Ly $\gamma$ forests. We further demonstrate an alternative method that treats the forest flux as a mixture of dark and transparent regions, where the latter are modeled using a physically-motivated parametric form for the intrinsic opacity distribution. The resulting model-dependent upper limits on $\xhi$ are similar to those derived from our fiducial model-independent analysis. We confirm that the bulk of reionization must be finished at $z>6$ , while leaving room for an extended "soft landing" to the reionization history down to $z\sim5.4$ suggested by Ly $\alpha$ forest opacity fluctuations.

</div>

<div id="div_fig1">

<img src="tmp_2510.25829/./xhi_thresh_allforest.png" alt="Fig8" width="100%"/>

**Figure 8. -** Measurements of the dark pixel fraction in each Lyman-series forest and forest combination inferred from the threshold method applied to the E-XQR-30 data set. Uncertainties (thick: 1$\sigma$, thin: 2$\sigma$) are derived from bootstrap resampling of the quasar sightlines contributing to each bin, or from the confidence interval of the binomial fraction, whichever is larger. For the forest combinations (lower panels), we show the dark pixel fractions of the individual contributing forests in a light color with slight redshift offsets for clarity. (*fig:dark_thresh*)

</div>
<div id="div_fig2">

<img src="tmp_2510.25829/./threshpix_abg.png" alt="Fig7.1" width="50%"/><img src="tmp_2510.25829/./threshpix_combo.png" alt="Fig7.2" width="50%"/>

**Figure 7. -** Distribution of detected (${\rm S/N}>2$, blue) and undetected (${\rm S/N}<2$, red) binned pixels in the $35$ E-XQR-30 quasar sightlines used in this work. White binned pixels within the low and high redshift boundaries of each quasar sightline have more than 50\% of their spectral pixels masked. The regions with faded color at the high redshift end of each sightline show pixels within the quasar proximity zone exclusion. Left: Binned pixels in the Ly$\alpha$(top), Ly$\beta$(middle), and Ly$\gamma$ forests. Right: Forest combinations Ly$\alpha+$Ly$\beta$(top), Ly$\beta+$Ly$\gamma$(middle), and Ly$\alpha+$Ly$\beta+$Ly$\gamma$(bottom), where blue pixels are detected in any or all forests while red pixels are undetected in all forests. (*fig:threshpix*)

</div>
<div id="div_fig3">

<img src="tmp_2510.25829/./xhi_thresh_fiducial.png" alt="Fig11.1" width="50%"/><img src="tmp_2510.25829/./xhi_negpix_fiducial.png" alt="Fig11.2" width="50%"/>

**Figure 11. -** Upper limits ($1\sigma$, thick; $2\sigma$, thin) on the IGM neutral fraction $\xhi$ from the dark pixel fractions estimated using the threshold method (left) and negative pixel method (right). In each redshift bin we show the fiducial forest/combination that has the lowest $2\sigma$ upper limit (see text). We compare our constraints to the dark pixel fraction measurements ($1\sigma$ upper limits) from \citet{McGreer15}(green points) and \citet{Jin23}(orange points), which were estimated using the negative pixel method and threshold method, respectively. (*fig:xhi_fiducial*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2510.25829"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$</div>



<div id="title">

# oMEGACat. VIII. A Subpopulation Census of $\omega$ Centauri

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2510.26341-b31b1b.svg)](https://arxiv.org/abs/2510.26341)<mark>Appeared on: 2025-10-31</mark> - 

</div>
<div id="authors">

<mark>C. Clontz</mark>, et al. -- incl., <mark>N. Neumayer</mark>

</div>
<div id="abstract">

**Abstract:** An understanding of the assembly history of the complex star cluster Omega Centauri has long been sought after, with many studies separating the stars on the color-magnitude diagram into multiple groupings across small magnitude ranges. Utilizing the oMEGACat combined astro-photometric and spectroscopic dataset we parse 14 subpopulations from the upper red-giant branch to below the main-sequence turnoff. We combine our results with previous works to estimate the age and age spread of each population. We find that the chemically enhanced (P2) populations are all $\sim$ 1 Gyr younger ( $\sim11.6$ Gyr old) and have significantly higher intrinsic age spreads (0.6 Gyr) than the primordial (P1) populations ( $\sim 12.6$ Gyr old, 0.3 Gyr spread), with the intermediate (Im) populations falling in between the two. Additionally, we connect for the first time the Chromosome Diagram to the two-stream age-metallicity relation, allowing us to link the P1 and P2 stars to the distinct star formation tracks, proposed to be in-situ and ex-situ contributions to the cluster's assembly. Our results are consistent with some suggested formation models and rule out others but no current model can explain all observed features of the subpopulations.

</div>

<div id="div_fig1">

<img src="tmp_2510.26341/./figures/sgb_rgb_population_labels.png" alt="Fig5" width="100%"/>

**Figure 5. -** **Subpopulation Parsing:**_(all panels)_ Each small marker point is a single star, colored by its subpopulation label. The large colored circle marks the associated cluster and within each is the annotation of the cluster number, in order of metallicity, as well as the ChD stream label, as a subscript. The 1- and 2-$\sigma$ density contours are shown in thick and thin black lines, respectively. _(upper left)_ The RGB chromosome map is constructed from the two delta-colors. The centroid of each subpopulation is given by the large colored markers. _(upper right)_ The pseudo-color vs. [Fe/H] space shows the distinction of the subpopulations in this space. _(lower panels)_ Same as upper panels except now for the SGB. Here, the large colored markers are offset to aid visibility and are connected to the centroid of each cluster via a solid line with the same color. (*fig:rgb_clustering*)

</div>
<div id="div_fig2">

<img src="tmp_2510.26341/./figures/two_panel_amr_age_spread.png" alt="Fig1" width="100%"/>

**Figure 1. -** **Subpopulation Age Relations**: ($_all panels_$) Each population is represented by a single medallion marker with the color corresponding to the cluster. Inside each the cluster number is annotated along with the relevant ChD stream indication subscript. The vertical black dashed lines show an approximate separation of the P1, Im and P2 populations. ($_upper panel_$) Black points mark all SGB stars with ages from [Clontz, Seth and Dotter (2024)](). Overplotted in the large colored circles are the mean (deconvolved) SGB age and median RGB metallicity for each subpopulation. The median age and [Fe/H] uncertainty for the SGB stars is given by the black contour while the median uncertainty on the mean age of each subpopulation is given by the black errorbars in the lower left. ($_lower panel_$) Each subpopulation mean age is plotted versus its intrinsic age spread. The median uncertainty on the mean age and intrinsic age spread is represented by a set of errorbars in the lower left corner. (*fig:amr_and_age_spread_two_panel*)

</div>
<div id="div_fig3">

<img src="tmp_2510.26341/./figures/cmd_with_regions_twopanel.png" alt="Fig3" width="100%"/>

**Figure 3. -** **Color-Magnitude Diagrams**:_(both panels)_ The black dotted line traces the overdense region of this plot when considering a subset of stars centered around the median metallicity ([Fe/H]$\sim$-1.7). _(left panel)_ The full subset of high-quality member stars is plotted with light grey markers. All SGB stars with age determinations are overplotted in black. The upper black box delineates the region where the RGB ChD is generated while the lower black box delineates where the age-metallicity relation is calculated. _(right panel)_ The full subset of high-quality member stars is plotted with individual markers colored by their [Fe/H] value, following the colorbar at the top of this panel. The black box delineates where the initial RGB clustering is performed. The upper grey shaded region shows the extent of the first propagation step centroiding sample while the lower grey shaded region shows the extent of the first propagation step assignment data. The scaling data is constituted by the combination of the centroiding and assignment data. (*fig:cmd_with_regions*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2510.26341"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\Rvis}{R_{550}}$
$\newcommand{\tauvis}{\tau_{550}}$
$\newcommand{\AV}{A_{\rm V}}$
$\newcommand{\micron}{\mu m}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\LiI}{\ion{Li}{i}}$
$\newcommand{\CaI}{\ion{Ca}{i}}$</div>



<div id="title">

# The dance of dust: Investigating young stellar object dipper variability $\thanks{Based on observations collected at the European Southern Observatory under ESO programmes 0105.C-0513(A),097.C-0378(A), 0101.C-0866(A), and 0103.C-0887(B).}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2510.26449-b31b1b.svg)](https://arxiv.org/abs/2510.26449)<mark>Appeared on: 2025-10-31</mark> -  _Accepted for publication in Astronomy & Astrophysics (A&A)_

</div>
<div id="authors">

E. A., et al.

</div>
<div id="abstract">

**Abstract:** The dipper subclass of young stellar objects (YSOs) are characterised by frequent dips in their light curves. Irregular dippers do not show periodic signatures and have dips accounting for significant proportions of their photospheric flux. Given the short timescales on which these dips occur, their driving mechanisms are linked to the inner circumstellar disc dynamics. We present the first multi-epoch analysis of 16 irregular dippers observed with X-Shooter. Investigating the properties of their dips, and in particular the analysis of the dust characteristics, we aim to understand the root of their variability, and get a glimpse of the inner disc behaviour. We employed a novel approach to measure the properties of the dips, by combining class III templates with Gaia multi-epoch photometry to construct the intrinsic photosphere of the objects. We measured several dip properties including the depth of the dips, near-infrared (NIR) excesses, and their optical depths as a function of wavelength. We record 20 significant dips that range in their dip properties and show no relation to one another. In almost all cases, the low optical depths and small NIR excesses are observed. Comparison of their optical depths with grain opacity models show that the dips can be explained by the presence of dust substructures containing processed grains obscuring their photospheres and/or their discs. These grain distributions can have maximum sizes as large as 20 $\mu m$ and in many cases have almost grey-like extinction, while some require a strong scattering component. The findings highlight the extent of the irregularity of dippers, but also link it to the dust dynamics in the inner regions of the circumstellar discs. The dust substructures causing the variability require processed dust grains to be lifted above the disc into the line of sight. Possible lifting mechanisms including disc winds, unstable accretion columns, and disc warps are discussed.

</div>

<div id="div_fig1">

<img src="tmp_2510.26449/./Plots/beta_abs_q3_colormap_half.png" alt="Fig3.1" width="25%"/><img src="tmp_2510.26449/./Plots/beta_sca_q3_colormap_half.png" alt="Fig3.2" width="25%"/><img src="tmp_2510.26449/./Plots/beta_ext_q3_colormap_half.png" alt="Fig3.3" width="25%"/><img src="tmp_2510.26449/./Plots/beta_eff_q3_colormap_half.png" alt="Fig3.4" width="25%"/>

**Figure 3. -** $\beta$ colour maps  from dust opacity models. Each figure represents a grid of dust size distributions varying  minimum and maximum grain sizes ($n(a)\propto a^{-q}$ with q=3.0). Coloured by the values of $\beta$ derived from the opacities for a) pure absorption ($\beta_{abs}$), b)scattering ($\beta_{sca}$, c) extinction ($\beta_{ext}$), and d) effective extinction ($\beta_{eff}$; see text). Contours are marked by black lines at $\beta$ =  0.0 and $\beta_{ISM} =1.38$). Pattern effects  are artefacts of the interpolation used.  (*fig: dustmaps*)

</div>
<div id="div_fig2">

<img src="tmp_2510.26449/./Plots/tau_masterplot.png" alt="Fig12" width="100%"/>

**Figure 12. -** $\tau_\lambda$ as a function of $\lambda$ for all the 20 dips analysed in Sect.\ref{sec: dust}. The name of the star and the epoch number are given on the secondary axis.
    The observed values are shown by black dots with error bars.  The red coloured lines show the results for the grain size distribution that best fit the data in the wavelength interval 400-900 nm for the extinction opacities, each normalised to the observed value at 550 nm. The values of $a_{min}, a_{max}, q$ are given on top of each panel; the corresponding  grain size distribution is displayed in the insert. The vertical grey dotted lines represent the boundaries of $\lambda = 400, 900$ nm. Three representative cases are also shown in the main paper. (*fig: master_tau*)

</div>
<div id="div_fig3">

<img src="tmp_2510.26449/./Plots/nir_excess.png" alt="Fig4" width="100%"/>

**Figure 4. -** Excess emission at 2.1 $\mu m$, $\epsilon_K$ shown as a function of $\Rvis$. The top panel plots the ratio between the observed flux and the intrinsic photospheric one ($\epsilon_K=\frac{F_{obs,K}}{F_{ph,K}}-1$). The bottom panel shows the ratio between the observed flux corrected for the extinction derived at optical wavelengths and the photospheric flux ($\epsilon_K=\frac{F_{obs,K}e^{\tau}}{F_{ph,K}}-1$). The grey area covers a $\pm$20\% region around $\epsilon_K$ = 0. Vertical error bars on $\epsilon_K$ are smaller than the size of the points. (*fig: nir_excess*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2510.26449"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

113  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

6  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
